<a href="https://colab.research.google.com/github/Hafizur-Rahman-SD/ML-with-Python-FCC-Course-/blob/main/Book_Recommendation_Engine_KNN_for_FCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import all required libraries
# These help in data loading, analysis, and KNN model training

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


In [5]:
# Download dataset directly from FreeCodeCamp server
# This ensures you get the correct version of the Book-Crossings dataset

!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip -O book-crossings.zip
!unzip -o book-crossings.zip

print("✅ Dataset downloaded and extracted successfully!")


--2025-10-06 14:16:55--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   151MB/s    in 0.2s    

2025-10-06 14:16:55 (151 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            
✅ Dataset downloaded and extracted successfully!
